<a href="https://colab.research.google.com/github/Ashish-kharde1/credit-card-fraud-detection/blob/main/credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import kagglehub
import pandas as pd
import numpy as np
import os

path = kagglehub.dataset_download("kartik2112/fraud-detection")
df = pd.read_csv(os.path.join(path, 'fraudTrain.csv'))
df.head()

Using Colab cache for faster access to the 'fraud-detection' dataset.


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [8]:
!pip install lightgbm imbalanced-learn shap

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [10]:
df.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06
mean,6.483370e+05,4.171920e+17,7.035104e+01,4.880067e+04,3.853762e+01,-9.022634e+01,8.882444e+04,1.349244e+09,3.853734e+01,-9.022646e+01,5.788652e-03
std,3.743180e+05,1.308806e+18,1.603160e+02,2.689322e+04,5.075808e+00,1.375908e+01,3.019564e+05,1.284128e+07,5.109788e+00,1.377109e+01,7.586269e-02
min,0.000000e+00,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902779e+01,-1.666712e+02,0.000000e+00
25%,3.241685e+05,1.800429e+14,9.650000e+00,2.623700e+04,3.462050e+01,-9.679800e+01,7.430000e+02,1.338751e+09,3.473357e+01,-9.689728e+01,0.000000e+00
50%,6.483370e+05,3.521417e+15,4.752000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.456000e+03,1.349250e+09,3.936568e+01,-8.743839e+01,0.000000e+00
75%,9.725055e+05,4.642255e+15,8.314000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.359385e+09,4.195716e+01,-8.023680e+01,0.000000e+00
max,1.296674e+06,4.992346e+18,2.894890e+04,9.978300e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.371817e+09,6.751027e+01,-6.695090e+01,1.000000e+00


In [11]:
df.is_fraud.value_counts()

,count
is_fraud,
0,1289169
1,7506


In [12]:
df.columns

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

In [13]:
df.drop(["cc_num","merchant","first","last","street","zip","trans_num","Unnamed: 0"], axis=1, inplace=True)

In [14]:
df.columns

Index(['trans_date_trans_time', 'category', 'amt', 'gender', 'city', 'state',
       'lat', 'long', 'city_pop', 'job', 'dob', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [15]:
df.dropna(inplace=True)

In [16]:
df.head()

,trans_date_trans_time,category,amt,gender,city,state,lat,long,city_pop,job,dob,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,misc_net,4.97,F,Moravian Falls,NC,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,grocery_pos,107.23,F,Orient,WA,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,entertainment,220.11,M,Malad City,ID,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,gas_transport,45.00,M,Boulder,MT,46.2306,-112.1138,1939,Patent attorney,1967-01-12,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,misc_pos,41.96,M,Doe Hill,VA,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,1325376186,38.674999,-78.632459,0


In [17]:
df.category.value_counts()

,count
category,
gas_transport,131659
grocery_pos,123638
home,123115
shopping_pos,116672
kids_pets,113035
shopping_net,97543
entertainment,94014
food_dining,91461
personal_care,90758


In [18]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [19]:
df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
df['age'] = (pd.Timestamp.now() - df['dob']).dt.days // 365
df.drop('dob', axis=1, inplace=True)

In [20]:
def haversine(lat1, lon1, lat2, lon2):
  R = 6371 #Earth radius in Km
  lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
  dlat = lat2 - lat1
  dlon = lon2 - lon1
  a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
  return 2 * R * np.arcsin(np.sqrt(a))

df['distance'] = haversine(df['lat'], df['long'], df['merch_lat'], df['merch_long'])

In [21]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'], format='%Y-%m-%d %H:%M:%S')

df['hour'] = df['trans_date_trans_time'].dt.hour
df['day'] = df['trans_date_trans_time'].dt.day
df['month'] = df['trans_date_trans_time'].dt.month
df['weekday'] = df['trans_date_trans_time'].dt.weekday

df.drop('trans_date_trans_time', axis=1, inplace=True)

In [22]:
cat_cols = ['category','gender','city','state','job']
encoder = LabelEncoder()
df["category"] = encoder.fit_transform(df["category"])
df["gender"] = encoder.fit_transform(df["gender"])
df["city"] = encoder.fit_transform(df["city"])
df["state"] = encoder.fit_transform(df["state"])
df["job"] = encoder.fit_transform(df["job"])

In [23]:
df.columns

Index(['category', 'amt', 'gender', 'city', 'state', 'lat', 'long', 'city_pop',
       'job', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud', 'age',
       'distance', 'hour', 'day', 'month', 'weekday'],
      dtype='object')

In [24]:
df.head()

,category,amt,gender,city,state,lat,long,city_pop,job,unix_time,merch_lat,merch_long,is_fraud,age,distance,hour,day,month,weekday
0,8,4.97,0,526,27,36.0788,-81.1781,3495,370,1325376018,36.011293,-82.048315,0,37,78.597568,0,1,1,1
1,4,107.23,0,612,47,48.8878,-118.2105,149,428,1325376044,49.159047,-118.186462,0,47,30.212176,0,1,1,1
2,0,220.11,1,468,13,42.1808,-112.2620,4154,307,1325376051,43.150704,-112.154481,0,63,108.206083,0,1,1,1
3,2,45.00,1,84,26,46.2306,-112.1138,1939,328,1325376076,47.034331,-112.561071,0,58,95.673231,0,1,1,1
4,9,41.96,1,216,45,38.4207,-79.4629,99,116,1325376186,38.674999,-78.632459,0,39,77.556744,0,1,1,1


In [25]:
target = 'is_fraud'
num_features = [
    'amt', 'lat', 'long', 'city_pop', 'merch_lat', 'merch_long', 'age', 'distance',
    'hour', 'day', 'month', 'weekday'
]
cat_features = [
    'category', 'gender', 'state', 'city', 'job'
]

In [26]:
num_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_features),
        ('cat', cat_pipeline, cat_features)
    ])

In [27]:
df.head()

,category,amt,gender,city,state,lat,long,city_pop,job,unix_time,merch_lat,merch_long,is_fraud,age,distance,hour,day,month,weekday
0,8,4.97,0,526,27,36.0788,-81.1781,3495,370,1325376018,36.011293,-82.048315,0,37,78.597568,0,1,1,1
1,4,107.23,0,612,47,48.8878,-118.2105,149,428,1325376044,49.159047,-118.186462,0,47,30.212176,0,1,1,1
2,0,220.11,1,468,13,42.1808,-112.2620,4154,307,1325376051,43.150704,-112.154481,0,63,108.206083,0,1,1,1
3,2,45.00,1,84,26,46.2306,-112.1138,1939,328,1325376076,47.034331,-112.561071,0,58,95.673231,0,1,1,1
4,9,41.96,1,216,45,38.4207,-79.4629,99,116,1325376186,38.674999,-78.632459,0,39,77.556744,0,1,1,1


In [28]:
X = df.drop(target, axis=1)
y = df[target]
X_preprocessed = preprocessor.fit_transform(X)

In [29]:
X.columns

Index(['category', 'amt', 'gender', 'city', 'state', 'lat', 'long', 'city_pop',
       'job', 'unix_time', 'merch_lat', 'merch_long', 'age', 'distance',
       'hour', 'day', 'month', 'weekday'],
      dtype='object')

In [30]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

model = lgb.LGBMClassifier()

In [31]:
train_data = lgb.Dataset(X_train, label=y_train)
val_data   = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [32]:
params = {
    "objective": "binary",
    "boosting_type": "gbdt",
    "learning_rate": 0.03,  # Decreased learning rate
    "num_leaves": 20,       # Reduced number of leaves
    "feature_fraction": 0.8,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "lambda_l1": 0.3, # Increased L1 regularization
    "lambda_l2": 0.3, # Increased L2 regularization
    "metric": ["auc"],
    "verbosity": -1,
    "is_unbalance": True,   # important for fraud detection
    "min_child_samples": 25 # Increased minimum child samples
}

In [33]:
from lightgbm import early_stopping, log_evaluation

model = lgb.train(
    params,
    train_data,
    num_boost_round=2000,  # Increased number of boosting rounds
    valid_sets=[train_data, val_data],
    valid_names=["train","val"],
    callbacks=[early_stopping(100), log_evaluation(100)] # Increased early stopping rounds
)

Training until validation scores don't improve for 100 rounds
[100]	train's auc: 0.997073	val's auc: 0.996735
[200]	train's auc: 0.99843	val's auc: 0.997928
[300]	train's auc: 0.998958	val's auc: 0.998307
[400]	train's auc: 0.999317	val's auc: 0.998591
[500]	train's auc: 0.999512	val's auc: 0.998771
[600]	train's auc: 0.999645	val's auc: 0.99891
[700]	train's auc: 0.999734	val's auc: 0.999003
[800]	train's auc: 0.999797	val's auc: 0.999073
[900]	train's auc: 0.999846	val's auc: 0.999159
[1000]	train's auc: 0.999881	val's auc: 0.999213
[1100]	train's auc: 0.999908	val's auc: 0.999263
[1200]	train's auc: 0.999928	val's auc: 0.999324
[1300]	train's auc: 0.999944	val's auc: 0.999374
[1400]	train's auc: 0.999956	val's auc: 0.999395
[1500]	train's auc: 0.999965	val's auc: 0.999442
[1600]	train's auc: 0.999973	val's auc: 0.999472
[1700]	train's auc: 0.999978	val's auc: 0.999502
[1800]	train's auc: 0.999982	val's auc: 0.999516
[1900]	train's auc: 0.999986	val's auc: 0.999541
[2000]	train's auc

In [34]:
from sklearn.metrics import precision_score, recall_score, fbeta_score, roc_auc_score, matthews_corrcoef, classification_report

# If you trained with lgb.train()
y_pred_proba = model.predict(X_test, num_iteration=model.best_iteration)  # gives probability of fraud (class=1)
y_pred = (y_pred_proba > 0.5).astype(int)  # threshold at 0.5



print("AUC:", roc_auc_score(y_test, y_pred_proba))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F2-score:", fbeta_score(y_test, y_pred, beta=2))
print("MCC:", matthews_corrcoef(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))

AUC: 0.9995608937443532
Precision: 0.6671251719394773
Recall: 0.9693537641572285
F2-score: 0.8888210140500916
MCC: 0.8028878709606034

Classification Report:
               precision    recall  f1-score   support

           0     0.9998    0.9972    0.9985    257834
           1     0.6671    0.9694    0.7903      1501

    accuracy                         0.9970    259335
   macro avg     0.8335    0.9833    0.8944    259335
weighted avg     0.9979    0.9970    0.9973    259335



In [35]:
import pickle

# Save the model
with open('lgb_model.pkl', 'wb') as f:
    pickle.dump(model, f)